<a href="https://colab.research.google.com/github/uday-codes69/pytorch-foundations/blob/main/Pytorch_rnn_based_Q_A_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

In [ ]:
df.head(4)

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter


In [ ]:
#tokenize
def tokenize(text):
  text = text.lower();
  text = text.replace('?','')
  text = text.replace(',','')
  text = text.replace('.','')
  text = text.replace('!','')
  return text.split()

In [ ]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [ ]:
#vocab
vocab = {'<UNK>' : 0}

In [ ]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)



In [ ]:
df.apply(build_vocab, axis = 1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 "'to": 12,
 'kill': 13,
 'a': 14,
 "mockingbird'": 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 "'1984'": 67,
 'george-orwell': 68,
 'currency': 69,
 '

In [ ]:
len(vocab)

326

In [ ]:
#convert words to numberical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [ ]:
text_to_indices('What is the capital of France?', vocab)

[1, 2, 3, 4, 5, 6]

In [50]:
import torch
from torch.utils.data import Dataset, DataLoader

In [52]:
class QADataset(Dataset):

  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
    pass

  def __len__(self):
    return self.df.shape[0]


  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [53]:
dataset = QADataset(df, vocab)

In [54]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [55]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[10, 75, 76]]) tensor([77])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[ 42,  18, 118,   3, 187, 188]]) tensor([189])
tensor([[ 42, 301, 302, 118,  14, 303, 304, 159, 305, 306, 307, 308]]) tensor([309])
tensor([[ 42, 137,   2, 138,  39, 176, 271]]) tensor([99])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([72])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[ 10,  96,   3, 104, 241]]) tensor([242])
tensor([[  1,   2,   3, 181, 182, 183, 184]]) tensor([185])
tensor([[ 42, 101,   2,   3,  17]]) tensor([102])
tensor([[ 42, 265, 266,  14, 267, 268, 159, 269]]) tensor([270])
tensor([[ 10,  11, 190, 159, 191]]) tensor([192])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 10, 140,   3, 141, 272,  93, 273,   5,   3, 274]]) tensor([275])
tensor([[ 78,  79, 151, 152,  14, 153, 154]]) tensor([155])
tensor([[ 42,   2,   3, 211, 137, 169, 212, 170]]) tensor([113])
tensor([[ 10,  75, 111]]) tensor([112])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
t

In [56]:
import torch.nn as nn

In [57]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [58]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [59]:
learning_rate = 0.001
epochs = 20

In [61]:
model = SimpleRNN(len(vocab))

In [62]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.043010
Epoch: 2, Loss: 452.973930
Epoch: 3, Loss: 372.182860
Epoch: 4, Loss: 314.788379
Epoch: 5, Loss: 264.593277
Epoch: 6, Loss: 217.565154
Epoch: 7, Loss: 175.121709
Epoch: 8, Loss: 137.777494
Epoch: 9, Loss: 108.048860
Epoch: 10, Loss: 83.250276
Epoch: 11, Loss: 64.361045
Epoch: 12, Loss: 50.477508
Epoch: 13, Loss: 40.605461
Epoch: 14, Loss: 33.034696
Epoch: 15, Loss: 27.316101
Epoch: 16, Loss: 22.592871
Epoch: 17, Loss: 19.033958
Epoch: 18, Loss: 16.241630
Epoch: 19, Loss: 14.007925
Epoch: 20, Loss: 12.059358


In [64]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [65]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [66]:
list(vocab.keys())[7]

'paris'